In [2]:
import pandas as pd
import numpy as np
import os
import re

### Data for Germany

In [3]:

# folder = r"C:\Users\jean-\Desktop\Cours ENSAE\3A ENSAE\ML for trading\Projet\ML-for-trading-project\Données projet\yield curve history Germany"

# # List all files in the folder
# files = os.listdir(folder)

# def extract_year(filename):
#     match = re.search(r'R(\d{2})XX', filename)
#     return match.group(1) if match else None

# # Create the dictionary of DataFrames with desired names
# yields_germany = {
#     f"yields Germany {extract_year(file)} years": pd.read_csv(os.path.join(folder, file),index_col =0)
#     for file in files if extract_year(file) is not None
# }

# list_years = [extract_year(file) for file in files if extract_year(file) is not None]

In [ ]:
# folder = r"./Données projet/yield curve history Germany"  # Use relative path for GitHub repo

# files = os.listdir(folder)

# def extract_year(filename):
#     match = re.search(r'R(\d{2})XX', filename)
#     return match.group(1) if match else None

# yields_germany = {
#     f"yields Germany {extract_year(file)} years": pd.read_csv(os.path.join(folder, file), index_col=0)
#     for file in files if extract_year(file) is not None
# }

# list_years = [extract_year(file) for file in files if extract_year(file) is not None]

In [ ]:

# start_date = '2000-01-01'
# end_date = '2025-10-13'

# business_days = pd.date_range(start=start_date, end=end_date, freq='B').strftime('%Y-%m-%d').tolist()

# for year in list_years:
#     df = yields_germany[f"yields Germany {year} years"]
#     series_filtered = df[df.index.isin(business_days)][f"BBSIS.D.I.ZST.ZI.EUR.S1311.B.A604.R{year}XX.R.A.A._Z._Z.A"]
#     df_filtered = pd.DataFrame(series_filtered)
#     df_filtered.columns = [f'{year}']
#     col_numeric = pd.to_numeric(df_filtered[f'{year}'], errors='coerce')
#     df_filtered = pd.DataFrame(col_numeric)
#     yields_germany[f"yields Germany {year} years"] = df_filtered[df_filtered[f'{year}'].notnull()]

    
   

### Data for US

For the US, we use the FRED MD database, available on the St Louis website, which contains a wide range of macroeconomic variables with monthly frequency.

In [10]:
df= pd.read_csv("./Données projet/data US/2025-09-MD.csv", index_col=0)

In [12]:
df.describe()

,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,IPCONGD,IPDCONGD,...,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,UMCSENTx,DTCOLNVHFNM,DTCTHFNM,INVEST,VIXCLSx
count,801.000000,801.000000,801.000000,8.000000e+02,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,...,801.000000,801.000000,801.000000,801.000000,801.000000,647.000000,800.000000,800.000000,801.000000,759.000000
mean,9676.729325,8232.720100,57.303749,8.303029e+05,223252.860662,67.965425,72.135863,70.924318,79.337440,67.042342,...,63.716896,54.591798,12.931473,15.004444,11.914769,84.690572,142905.059450,359727.393038,1378.618822,19.278566
std,5178.645586,4100.614248,31.316873,3.904708e+05,192090.783435,27.032184,27.292419,27.570917,23.533362,28.789153,...,31.524712,35.020187,8.175654,9.183192,7.335325,13.659669,146268.035165,341935.326025,1538.347702,7.075780
min,5.000000,5.000000,5.000000,5.000000e+00,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,6.000000,6.000000,2.130000,2.450000,2.040000,2.000000,6.000000,6.000000,6.000000,1.000000
25%,5148.700000,4583.400000,29.110000,4.892787e+05,50436.918730,45.022100,47.154300,44.920000,61.377200,42.258000,...,33.313000,19.436000,5.170000,6.810000,4.780000,74.200000,10828.250000,34319.000000,192.315600,14.294950
50%,8487.783000,7313.200000,49.254000,7.239413e+05,159921.000000,62.919900,71.763800,69.964400,81.760700,61.828000,...,67.390000,52.481000,11.960000,13.760000,11.360000,88.600000,81295.500000,190899.875000,756.001700,17.621600
75%,13789.923000,11624.900000,84.409000,1.185876e+06,355792.000000,95.929400,99.454400,98.730900,100.669600,98.204900,...,93.328000,84.159000,19.710000,22.390000,17.990000,94.700000,269654.840000,731629.750000,2143.175800,22.513350
max,22918.422000,16739.900000,124.805000,1.565742e+06,732010.000000,104.211500,110.212000,108.675800,111.757500,109.816700,...,120.620000,133.209000,32.560000,37.470000,29.030000,112.000000,559708.930000,949306.220000,5846.707800,62.964800


In [18]:
pd.set_option('display.max_seq_items', None)  # pas de "..." pour les Index/Series
pd.set_option('display.max_columns', None)    # affiche toutes les colonnes
print(df.columns)

Index(['RPI', 'W875RX1', 'DPCERA3M086SBEA', 'CMRMTSPLx', 'RETAILx', 'INDPRO',
       'IPFPNSS', 'IPFINAL', 'IPCONGD', 'IPDCONGD', 'IPNCONGD', 'IPBUSEQ',
       'IPMAT', 'IPDMAT', 'IPNMAT', 'IPMANSICS', 'IPB51222S', 'IPFUELS',
       'CUMFNS', 'HWI', 'HWIURATIO', 'CLF16OV', 'CE16OV', 'UNRATE', 'UEMPMEAN',
       'UEMPLT5', 'UEMP5TO14', 'UEMP15OV', 'UEMP15T26', 'UEMP27OV', 'CLAIMSx',
       'PAYEMS', 'USGOOD', 'CES1021000001', 'USCONS', 'MANEMP', 'DMANEMP',
       'NDMANEMP', 'SRVPRD', 'USTPU', 'USWTRADE', 'USTRADE', 'USFIRE',
       'USGOVT', 'CES0600000007', 'AWOTMAN', 'AWHMAN', 'HOUST', 'HOUSTNE',
       'HOUSTMW', 'HOUSTS', 'HOUSTW', 'PERMIT', 'PERMITNE', 'PERMITMW',
       'PERMITS', 'PERMITW', 'ACOGNO', 'AMDMNOx', 'ANDENOx', 'AMDMUOx',
       'BUSINVx', 'ISRATIOx', 'M1SL', 'M2SL', 'M2REAL', 'BOGMBASE', 'TOTRESNS',
       'NONBORRES', 'BUSLOANS', 'REALLN', 'NONREVSL', 'CONSPI', 'S&P 500',
       'S&P div yield', 'S&P PE ratio', 'FEDFUNDS', 'CP3Mx', 'TB3MS', 'TB6MS',
       'GS1', 'G

### Extracting data with the API

In [23]:
pip install fredapi


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 25.2
[notice] To update, run: C:\Users\jean-\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [24]:
api_key = '615add3bd441c0ed42dec47e952d69e2'
from fredapi import Fred
fred = Fred(api_key=api_key)
print(fred.get_series('GDP').tail())

2024-04-01    29147.044
2024-07-01    29511.664
2024-10-01    29825.182
2025-01-01    30042.113
2025-04-01    30485.729
dtype: float64


In [ ]:


variables_to_keep = [#production and revenue variables
    'RPI',  #Real personal income; monitored by FED for interest rates decisions
                   'INDPRO', #Industrial production index; leading indicator of economic health, monitored by FED 
                   #job market variables
                   'UNRATE', #unemployment rate
                   'UEMP5TO14', #unemployment for people unemployed between 5 and 14 weeks = short term unemployment
                   'UEMP27OV', #unemployment for people unemployed for 27 weeks and over = long term unemployment
                    'PAYEMS', #total non farm payroll, leading indicator of employment
                    'USGOOD',   #below is the breakdown of employment figures for various economic sectors
                    'CES1021000001',
                    'USCONS',
                    'MANEMP',
                    'DMANEMP',
                    'NDMANEMP',
                    'SRVPRD',
                    'USTPU',
                    'USWTRADE',
                    'USTRADE',
                    'USFIRE',
                    'USGOVT',
                    #consumption and housing
                    'HOUST', #housing variable, sensitive to interest rates and economic anticipations
                    'PERMIT', #same but for building permits
                    'UMCSENT', #university of michigan consumer sentiment index, important for consumption anticipations
                    #money and credit variables
                    'M1SL', #monetary base, important for interest rates and inflation anticipations
                    'M2SL', 
                    'M2REAL', #real monetary base
                    'TOTRESNS', #total reserves of banks
                    'NONBORRES', #non borrowable reserves of banks
                    'BUSLOANS', #business loans, important for growth anticipations
                    'REALLN', #real loans, related to economic activity
                    'NONREVSL', #non-revolving consumer credit, related to consumption
                    'DTCOLNVHFNM', #total nonfinancial corporate business liabilities, related to economic activity
                    'DTCTHFNM', #total household liabilities, related to consumption
                    'INVEST', #gross private domestic investment, related to economic activity
                    #price indexes
                    'CPIAUCSL', #principal measure for inflation monitored by FED
                    'CPIAUCSL', 'CPIAPPSL', 'CPITRNSL','CPIMEDSL', 'CUSR0000SAC', 'CUUR0000SAD', 'CUSR0000SAS', 'CPIULFSL', 'CUUR0000SA0L2', 'CUSR0000SA0L5', #breakdown of the index for various sectors
                    'PCEPI', #inflation target for FED
                    'DDURRG3M086SBEA', 'DNDGRG3M086SBEA', 'DSERRG3M086SBEA', #spendings for services, durable and nondurable goods
                    #fiscal variables
                    'FGEXPND', #quarterly federal government expenditures
                    'FGRECPT', #quarterly federal government receipts
                    'FGDEF',   #quarterly federal government budget deficit
                    'FGCE', # quarterly federal government spending
                    'MTSDS133FMS', # monthly federal surplus or deficit
                    'FYGFDPUN', #federal debt held by the public, quarterly 
                     'GFDEGDQ188S', #debt to gdp ratio, quarterly 
                     'A091RC1Q027SBEA', #net interest payment, quarterly 
                     'GFDEBTN', #total debt outstanding, monthly 
                     #yield variables
                     'DGS1MO', 'DGS3MO', 'DGS6MO', #yields on treasury bills with maturities < 1y
                    'DGS1', 'DGS2', 'DGS3','DGS5','DGS7', 'DGS10', 'DGS20', 'DGS30', #yields of government treasuries for maturies >=1y
                    #'GS1M','TB3MS','TB6MS', #yield on commercial paper
                    #'GS1', 'GS2', 'GS3','GS5','GS7', 'GS10', 'GS20', 'GS30', #yields of government treasuries for maturies >=1y  
                    #yields of other "safe haven"
                    'IRLTLT01DEM156N', #10y german bund yield
                    'IRLTLT01JPM156N', #10y japanese gov yield
                    'IRLTLT01GBM156N', #10y uk gilt yield
                    'IRLTLT01CAM156N', #10y canadian gov yield
                    'IRLTLT01AUM156N', #10y australian gov yield
                    'IRLTLT01FRM156N', #10y french gov yield
                    #stock markets
                    'SP500', 
                    'NASDAQCOM', #nasdaq
                    #USD FX rates and US corporate bonds variables
                    'AAA', 'BAA', #yields of corporate bonds with notations AAA and BAA
                    'DEXUSEU', #usd to euro exchange rate
                    'DEXJPUS', #usd to yen exchange rate
                    'DEXUSUK', #usd to gbp exchange rate
                    'DEXCAUS', #usd to cad exchange rate
                    'DEXUSAL', #usd to aud exchange rate
                    'DEXCHUS', #usd to yuan exchange rate
                    #other financial variables measuring risk on / risk off 
                    'VIXCLS', #vix index, measure of market volatility
                    'NFCI', #national financial conditions index
                    'DCOILWTICO', #crude oil price, important for inflation anticipation
                     #interest rates variables (traditional monetary policy)
                    'FEDFUNDS', #federal funds rate, short term interest rates
                    'IORB', #interest on reserves
                    'OBFR', #overnight funding rate
                    'SOFR', #secured overnight financing rate
                    'WRESBAL', #measure of liquidity in the banking system
                    'RRPONTSYD', #overnight reverse repurchase agreement, measure of liquidity in the banking system
                    'CURRCIR', #currency in circulation
                    'WTREGEN', #government deposits 
                    #QE and QT variables 
                    'BOGMBASE', #monetary base
                    'WSHOSHO', #total assets of FED = size of balance sheet
                    'WSHOTSL', #total treasuries held by FED
                    'WSHOBL', #total longer duration trasuries held by FED (for long end of yield curve)
                    #anticipations of inflation by markets
                    'T5YIE', #5y forward inflation expectation
                    'T10YIE', #10y forward inflation expectation

                    ]




start_date = '2003-01-02' #we don't do a start date because lots of data are missing before that 
end_date   = '2025-08-01'


In [48]:
data = {}
for code_variable in variables_to_keep:
    print(code_variable)
    try:
        series = fred.get_series(code_variable, observation_start=start_date, observation_end=end_date)
        data[code_variable] = series
    except ValueError:
        print(f"Could not retrieve data for {code_variable}")
        continue

RPI
INDPRO
UNRATE
UEMP5TO14
UEMP27OV
PAYEMS
USGOOD
CES1021000001
USCONS
MANEMP
DMANEMP
NDMANEMP
SRVPRD
USTPU
USWTRADE
USTRADE
USFIRE
USGOVT
HOUST
PERMIT
ACOGNO
UMCSENT
M1SL
M2SL
M2REAL
AMBSL
TOTRESNS
NONBORRES
BUSLOANS
REALLN
NONREVSL
MZMSL
DTCOLNVHFNM
DTCTHFNM
INVEST
PPIFGS
PPIFCG
PPICRM
PPICMM
CPIAUCSL
CPIAUCSL
CPIAPPSL
CPITRNSL
CPIMEDSL
CUSR0000SAC
CUUR0000SAD
CUSR0000SAS
CPIULFSL
CUUR0000SA0L2
CUSR0000SA0L5
PCEPI
DDURRG3M086SBEA
DNDGRG3M086SBEA
DSERRG3M086SBEA
FGEXPND
FGRECPT
FGDEF
FGCE
MTSDS133FMS
FYGFDPUN
GFDEGDQ188S
A091RC1Q027SBEA
GFDEBTN
GS1M
TB3MS
TB6MS
GS1
GS2
GS3
GS5
GS7
GS10
GS20
GS30
IRLTLT01DEM156N
IRLTLT01JPM156N
IRLTLT01GBM156N
IRLTLT01CAM156N
IRLTLT01AUM156N
IRLTLT01FRM156N
SP500
NASDAQCOM
DJIA
AAA
BAA
DTWEXM
GOLDPMGBD228N
Could not retrieve data for GOLDPMGBD228N
VIXCLS
STLFSI2
NFCI
DCOILWTICO
FEDFUNDS
IORB
OBFR
SOFR
WRESBAL
RRPONTSYD
CURRCIR
WTREGEN
BOGMBASEW
WSHOSHO
WSHOTSL
WSHOBL
T5YIE
T10YIE


In [74]:
for code in variables_to_keep:
    info = fred.get_series_info(code)
    print(f"Code: {code}",f"Title / Description: {info['title']}",f"Frequency: {info['frequency']}",f"Observation End: {info['observation_end']}")
    

Code: RPI Title / Description: Real Personal Income Frequency: Monthly Observation End: 2025-08-01
Code: INDPRO Title / Description: Industrial Production: Total Index Frequency: Monthly Observation End: 2025-08-01
Code: UNRATE Title / Description: Unemployment Rate Frequency: Monthly Observation End: 2025-08-01
Code: UEMP5TO14 Title / Description: Number Unemployed for 5-14 Weeks Frequency: Monthly Observation End: 2025-08-01
Code: UEMP27OV Title / Description: Number Unemployed for 27 Weeks & over Frequency: Monthly Observation End: 2025-08-01
Code: PAYEMS Title / Description: All Employees, Total Nonfarm Frequency: Monthly Observation End: 2025-08-01
Code: USGOOD Title / Description: All Employees, Goods-Producing Frequency: Monthly Observation End: 2025-08-01
Code: CES1021000001 Title / Description: All Employees, Mining, Quarrying, and Oil and Gas Extraction Frequency: Monthly Observation End: 2025-08-01
Code: USCONS Title / Description: All Employees, Construction Frequency: Mont

In [79]:
code = 'BOGMBASESA'
info = fred.get_series_info(code)
print(f"Code: {code}")
print(f"Title / Description: {info['title']}")
print(f"Frequency: {info['frequency']}")
print(f"Units: {info['units']}")
print(f"Observation Start: {info['observation_start']}")
print(f"Observation End: {info['observation_end']}")

    

ValueError: Bad Request.  The series does not exist.